In [168]:
import random
import secrets

In [169]:
from sympy import mod_inverse

In [170]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

In [171]:
def RMPT(p, s): #Assuming number of bits > 600
    q = p - 1
    k = 0
    while q % 2 != 1:
        q //= 2
        k += 1
        
    for _ in range(s):
        a = random.randrange(3, p - 2)      
        while gcd(a, p) != 1:
            a = random.randrange(3, p - 2)
        
        z = pow(a, q, p)
        
        if z == 1 or z == p - 1:
            continue #Not a witness, pick another a
        
        for i in range(1, k):
            z = pow(a, pow(2, i)*q, p)
            if z == p - 1:
                break #Not a witness, pick another a
        else:
            return False  #a^((2^K)*Q)modP != -1 for all values of K, it is a witness, therefore its definitely not prime
        
    return True #After S iterations still not a composite, chance to be a prime

In [172]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [173]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

In [174]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = mod_inverse(e, phi_N)
    return d

In [175]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

In [176]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

Demo of the attack

Preparation of variables

In [177]:
p = Generate_Prime(1024, 15)

83135304480932928942168875954057172669167629731266225719121833672659593610683098722483392288638251183533255916083909504356799787213684032699426547980888502612610829295840272444831958786840845554678631696016626684958575952834632972274324144920572410847584855254344714670384829393774843463667184926044729491947


In [178]:
q = Generate_Prime(1024, 15)

79673348221730066089257079429257407421836567088442696855654865335794171917724581636229017911221527541660893938279192584370665281244365064394631360847644216411189042759864709080384168476659209166368176851446237875260808759667739161636347492296599675717834919811266189299126241255622191125784720953431699460777


In [179]:
N = p * q

In [180]:
e1=17

In [181]:
e2=257

In [182]:
d1 = RSA_generate_D(e1, p, q)

In [183]:
d2 = RSA_generate_D(e2, p, q)

In [184]:
message = "For this operation to succeed, both of you need to be mobilized simultaneously. You'll meet your partner once you get there."

ascii_message = convert_to_ascii(message)
print(ascii_message)
ascii_message.bit_length()

70111114032116104105115032111112101114097116105111110032116111032115117099099101101100044032098111116104032111102032121111117032110101101100032116111032098101032109111098105108105122101100032115105109117108116097110101111117115108121046032089111117039108108032109101101116032121111117114032112097114116110101114032111110099101032121111117032103101116032116104101114101046


1232

Encryptions

In [185]:
encrypted_message_1 = RSA_encrypt(ascii_message, e1, N)

In [186]:
encrypted_message_2 = RSA_encrypt(ascii_message, e2, N)

Decryptions

In [187]:
decrypted_message_1 = RSA_encrypt(encrypted_message_1, d1, N)

In [188]:
decrypted_message_2 = RSA_encrypt(encrypted_message_2, d2, N)

Calculating a1 and a2

In [189]:
a1=mod_inverse(e1,e2)

In [190]:
a2=int((1-(e1*a1))/e2)

Attempt to decrypt: encrypted_message_1^a1 mod N

In [191]:
decrypt_attempt_1=(pow(encrypted_message_1,a1,N))

Attempt to decrypt: encrypted_message_2^a2 mod N. Since a2 is negative, we inverse this message to mod N and power it with (-a2)

In [192]:
encrypt_inverse=mod_inverse(encrypted_message_2,N)

In [193]:
decrypt_attempt_2=(pow(encrypt_inverse,(-a2),N))

Multiply them together

In [194]:
decrypt_attempt=(decrypt_attempt_1*decrypt_attempt_2)%N

Check decrypted messages of both recipients

In [195]:
convert_to_string(decrypted_message_1)

"For this operation to succeed, both of you need to be mobilized simultaneously. You'll meet your partner once you get there."

In [196]:
convert_to_string(decrypted_message_2)

"For this operation to succeed, both of you need to be mobilized simultaneously. You'll meet your partner once you get there."

Did the eavesdropping succeed?

In [197]:
convert_to_string(decrypt_attempt)

"For this operation to succeed, both of you need to be mobilized simultaneously. You'll meet your partner once you get there."